## Part 2: Data Wrangling and Analysis

## Step 4: Data Wrangling

We'll start off by connecting to the database, import pandas as pd, and then load back the DataFrames: airlines, airports, routes, incidents and flights from SQL

In [1]:
import sqlite3
import pandas as pd

engine = sqlite3.connect('HW1_DB')

In [2]:
airlines_df = pd.read_sql_query('SELECT * FROM airlines', engine)
airports_df = pd.read_sql_query('SELECT * FROM airports', engine)
routes_df = pd.read_sql_query('SELECT * FROM routes', engine)
incidents_df = pd.read_sql_query('SELECT * FROM incidents', engine)
flights_df = pd.read_sql_query('SELECT * FROM flights', engine)

## Step 4.1: Understanding and inspecting routes_df

Recall details of `routes_df`...

In [3]:
routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66548 entries, 0 to 66547
Data columns (total 9 columns):
airline             66548 non-null object
airline_id          66548 non-null int64
src_iata_icao       66548 non-null object
source_id           66548 non-null int64
target_iata_icao    66548 non-null object
target_id           66548 non-null int64
code_share          66548 non-null object
stops               66548 non-null int64
equipment           66548 non-null object
dtypes: int64(4), object(5)
memory usage: 4.6+ MB


In [4]:
routes_df.head(15)

,airline,airline_id,src_iata_icao,source_id,target_iata_icao,target_id,code_share,stops,equipment
0,2B,410,AER,2965,KZN,2990,,0,CR2
1,2B,410,ASF,2966,KZN,2990,,0,CR2
2,2B,410,ASF,2966,MRV,2962,,0,CR2
3,2B,410,CEK,2968,KZN,2990,,0,CR2
4,2B,410,CEK,2968,OVB,4078,,0,CR2
5,2B,410,DME,4029,KZN,2990,,0,CR2
6,2B,410,DME,4029,NBC,6969,,0,CR2
7,2B,410,DME,4029,UUA,6160,,0,CR2
8,2B,410,EGO,6156,KGD,2952,,0,CR2
9,2B,410,EGO,6156,KZN,2990,,0,CR2


In [5]:
routes_df.describe()

,airline_id,source_id,target_id,stops
count,66548.000000,66548.000000,66548.000000,66548.000000
mean,3367.569754,2660.201929,2662.088192,0.000165
std,3182.024330,1577.965637,1578.661433,0.012856
min,10.000000,1.000000,1.000000,0.000000
25%,1317.000000,1382.000000,1382.000000,0.000000
50%,2942.000000,2958.000000,2958.000000,0.000000
75%,4608.000000,3670.000000,3670.000000,0.000000
max,19676.000000,9327.000000,9327.000000,1.000000


### 4.1.1 Computing Flight Routes

Now suppose we are interested in what flights are available out of PHL or JFK.  

We'll compute this using `routes_df`, showing just the airline (`airline_iata`), source (`src_iata_icao`) and target (`target_iata_icao`). 

To make it easier to understand, we'll also order the result primarily by `src_iata_icao` and secondarily by `target_iata_icao`.

In [6]:
JFK_PHL_df = routes_df.query('src_iata_icao == "JFK" | src_iata_icao == "PHL"').\
loc[:, ['airline_id', 'src_iata_icao', 'target_iata_icao']].sort_values(by=['src_iata_icao', 'target_iata_icao'])

In [7]:
# View JFK_PHL_df
JFK_PHL_df.head(20)

,airline_id,src_iata_icao,target_iata_icao
13675,3029,JFK,ABQ
20490,2009,JFK,ACC
5231,24,JFK,AMM
47125,4259,JFK,AMM
58517,5265,JFK,AMM
20491,2009,JFK,AMS
36079,3090,JFK,AMS
5232,24,JFK,ANU
58518,5265,JFK,ANU
22050,3737,JFK,ARN


### 4.1.2 Busy Airports

Now let's understand more about the really busy airports in `routes_df`.  To do this, we'll first create a dataframe `busy_df` which shows the number of flights out of each source airport, restricted to those with more than 500 such flights.

The result should be indexed by `src_iata_icao` with an aggregate field labeled `count` - order the result so that the busiest airport is first.

In [8]:
busy_df = routes_df.groupby('src_iata_icao').size().reset_index(name = 'count').sort_values('count', ascending = False)
busy_df = busy_df[busy_df['count'] > 500].set_index('src_iata_icao')

In [9]:
# View busy_df
busy_df.head(20)

,count
src_iata_icao,
ATL,909
ORD,558
PEK,527
LHR,527
CDG,524


## 4.2 Understanding airline information

Now let's understand the *airline* information within the datasets. 

### 4.2.1 Airlines and Flights
Start by recalling the details of `airlines_df` and `flights_df`, after extracting `flights_df` from the tables.

In [10]:
flights_df = pd.read_sql_query('SELECT * FROM flights', engine)

### 4.2.2 Airline Computations

We'll calculate the following information about airlines in the datasets.

- active_airlines: the number of active airlines in `airlines_df` 

- flights_airlines:  the number of different airlines in `flights_df`

- missing_flights:  the number of *active* airlines in `airlines_df` that are not in `flights_df`

In [11]:
import numpy as np

results = {'active_airlines': sum(airlines_df['active'] == 'Y'),
          'flights_airlines': len(flights_df['FL_NUM'].unique()),
          'missing_flights': len(np.setdiff1d(airlines_df[airlines_df['active'] == 'Y']['airline_id'], flights_df['FL_NUM'].unique()))}


In [12]:
# View results
results

{'active_airlines': 1161, 'flights_airlines': 6966, 'missing_flights': 421}

### 4.2.3 "Risky" Airlines

Using `incidents_df`, we want to understand the “risky” airlines. To do this, we'll calculate the frequency of occurrence of each airline in `incidents_df`, and choose the top-10.

In [13]:
incidents_df = pd.read_sql_query('SELECT * FROM incidents', engine)

In [14]:
# Calculate top-10 "risky" airlines
# Put result in a DataFrame called risky_df 
# Index df by rank (0 being the most frequent), and having columns 'Airline' and 'count'.

risky_df = incidents_df.groupby('Airline').size().reset_index(name = 'count').\
sort_values('count', ascending = False).head(10).reset_index(drop = True)

In [15]:
# View risky_df
risky_df

,Airline,count
0,American Airlines,6
1,Turkish Airlines,6
2,Southwest Airlines,5
3,Air France,5
4,Merpati Nusantara Airlines,5
5,EgyptAir,5
6,British Airways,4
7,China Airlines,4
8,Qantas,4
9,FedEx Express,4


### 4.2.4 Delayed and Cancelled

Now we're going to calculate *all* airlines with delayed or cancelled flights, i.e. the airlines of flights in `flights_df` whose arrival was delayed by 30 min or more or which were cancelled.  We will call this `delayed_cancelled_df`.

In [16]:
delayed_cancelled_df = flights_df.query('ARR_DELAY_NEW >= 30.0 or CANCELLED == 1.0').groupby('CARRIER').size().\
reset_index(name = 'count').sort_values('count', ascending = False).reset_index(drop = True)

In [17]:
# View the dataframe
delayed_cancelled_df

,CARRIER,count
0,WN,11762
1,OO,11093
2,AA,8186
3,DL,7910
4,B6,6447
5,UA,4778
6,OH,4657
7,YX,4303
8,EV,3896
9,MQ,3870


## Step 5: Combining DataFrames 

In this part, we want to combine information about airlines from both sources, `risky_df` (pulled from `incidents_df`) and `delayed_cancelled_df` (pulled from `flights_df`) to calculate the “very bad” airlines.  These are airlines that are both “risky” and which are “bad flights.”   This is an example of **data integration**, i.e. of linking data across heterogeneous sources.

It is helpful to develop **schema matching** techniques to help discover these correspondences. 

## 5.1 Relating Airlines and Flights: Schema Matching

Let's start by detecting what fields (columns) “match” between the `airlines_df` and `flights_df` DataFrames.  

To measure this in a principled way, we’ll use a measure called the Jaccard distance (or Jaccard index or Jaccard measure).  The Jaccard distance measures similarity between two sets A and B, and is merely:

\begin{equation*}
| A \cap B | / |A \cup B|
\end{equation*}


### 5.1.1 Column / Schema Matches

In [18]:
# Find what columns relate in airlines_df and flights_df 
# Create nested map of Jaccard distance

best_match = {}


# YOUR CODE HERE
for outerkey in flights_df.columns:
    if flights_df[outerkey].dtype == 'object':
        dict1 = {}
        dict2 = {}
        for innerkey in airlines_df.columns:
            if airlines_df[innerkey].dtype == 'object':
                flights_clean = flights_df.drop_duplicates(subset = outerkey)
                airlines_clean = airlines_df.drop_duplicates(subset = innerkey)
                
                flights_array = flights_clean[outerkey]
                airlines_array = airlines_clean[innerkey]
                
                value = len(np.intersect1d(flights_array, airlines_array)) / len(flights_array.append(airlines_array))

                dict3 = dict(key = value)
                dict3[innerkey] = dict3.pop('key')
                #print(dict3)
                
                dict2.update(dict3)
                #print(dict2)
                
            dict1 = dict(key = dict2)
            dict1[outerkey] = dict1.pop('key')
            #print(dict1)
        
        best_match.update(dict1)

In [19]:
# View the dictionary
best_match

{'CARRIER': {'airline_name': 0.0,
  'alias': 0.0,
  'iata': 0.016245487364620937,
  'icao': 0.0,
  'airline_callsign': 0.0,
  'country': 0.0,
  'active': 0.0},
 'ORIGIN': {'airline_name': 0.0,
  'alias': 0.0,
  'iata': 0.0,
  'icao': 0.03459016393442623,
  'airline_callsign': 0.0005406379527842854,
  'country': 0.0,
  'active': 0.0},
 'DEST': {'airline_name': 0.0,
  'alias': 0.0,
  'iata': 0.0,
  'icao': 0.03459016393442623,
  'airline_callsign': 0.0005406379527842854,
  'country': 0.0,
  'active': 0.0}}

### 5.1.2 Matching columns

The above map should help decide which column in `airlines_df` is the best match for `CARRIER` in `flights_df`.

In [20]:
my_choice = 'iata'

mapping = {'CARRIER': my_choice} 

## 5.2  Joining DataFrames

Now that we know how `airports_df` and `flights_df` relate, we'll use this to add the names of airports to `delayed_cancelled_df`.

In [21]:
# Create joint DataFrame delayed_cancelled_ext_df.  

delayed_cancelled_ext_df = delayed_cancelled_df.merge(airlines_df, left_on = 'CARRIER', right_on = 'iata').\
loc[:, ['airline_name', 'airline_id', 'count']]

In [22]:
# View delayed_cancelled_ext_df
delayed_cancelled_ext_df

,airline_name,airline_id,count
0,Southwest Airlines,4547,11762
1,SkyWest,4738,11093
2,American Airlines,24,8186
3,Delta Air Lines,2009,7910
4,JetBlue Airways,3029,6447
5,United Airlines,5209,4778
6,Comair,1828,4657
7,Midwest Airlines,3497,4303
8,Atlantic Southeast Airlines,452,3896
9,American Eagle Airlines,659,3870


## 5.3 Calculate "very bad" airlines

We are finally in a position to calculate the “very bad” airlines, i.e. those that are in both `risky_df` and `delayed_cancelled_ext_df`. We'll do this by merging the two dataFrames to create a result.

In [23]:
risky_df.rename(columns = {'count': 'accident_count'}, inplace = True)
delayed_cancelled_ext_df.rename(columns = {'count': 'dc_count'}, inplace = True)

In [24]:
# Merge the two DataFrames to create a result that has airline_name, as well as accident_count and dc_count.

very_bad_df = pd.merge(risky_df, delayed_cancelled_ext_df, left_on = 'Airline', right_on = 'airline_name')
very_bad_df = very_bad_df[['airline_name', 'accident_count', 'dc_count']]

In [25]:
# View very_bad_df
very_bad_df

,airline_name,accident_count,dc_count
0,American Airlines,6,8186
1,Southwest Airlines,5,11762
